In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.model_selection import train_test_split

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [283]:
housing = pd.read_csv("housing.csv")
housing.head()

,0.00632,18,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24
0,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
1,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
2,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
3,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
4,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21,28.7


In [284]:
housing.corr()

,0.00632,18,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24
0.00632,1.000000,-0.200283,0.406251,-0.056132,0.420934,-0.218978,0.352701,-0.379626,0.625395,0.582568,0.289393,-0.384838,0.455328,-0.388249
18,-0.200283,1.000000,-0.534022,-0.042550,-0.516574,0.311835,-0.569524,0.664396,-0.311717,-0.314351,-0.391713,0.175319,-0.412894,0.360393
2.31,0.406251,-0.534022,1.000000,0.062350,0.764556,-0.391330,0.645543,-0.708848,0.594167,0.720561,0.380955,-0.356506,0.602737,-0.484126
0,-0.056132,-0.042550,0.062350,1.000000,0.091134,0.091497,0.086461,-0.099109,-0.007907,-0.035965,-0.122570,0.049040,-0.054576,0.175364
0.538,0.420934,-0.516574,0.764556,0.091134,1.000000,-0.302127,0.731461,-0.769220,0.611758,0.668141,0.188918,-0.380006,0.591262,-0.427295
6.575,-0.218978,0.311835,-0.391330,0.091497,-0.302127,1.000000,-0.240211,0.205170,-0.209277,-0.291680,-0.355116,0.127754,-0.613734,0.695365
65.2,0.352701,-0.569524,0.645543,0.086461,0.731461,-0.240211,1.000000,-0.747872,0.456232,0.506527,0.261724,-0.273486,0.602782,-0.376932
4.09,-0.379626,0.664396,-0.708848,-0.099109,-0.769220,0.205170,-0.747872,1.000000,-0.494797,-0.534492,-0.232560,0.291451,-0.497276,0.249895
1,0.625395,-0.311717,0.594167,-0.007907,0.611758,-0.209277,0.456232,-0.494797,1.000000,0.910202,0.463322,-0.444065,0.487608,-0.381690
296,0.582568,-0.314351,0.720561,-0.035965,0.668141,-0.291680,0.506527,-0.534492,0.910202,1.000000,0.460100,-0.441505,0.543435,-0.468543


In [275]:
class LinearRegression:
    def __init__(self, X, y, learningrate, tolerance, maxIteration = 50000, error = 'rmse', gd = False, 
                regression = False, stochastic = False, batch_size = 100, regularize = False, regLambda = 1):
        self.X = X
        self.y = y
        self.learningrate = learningrate
        self.tolerance = tolerance
        self.maxIteration = maxIteration
        self.error = error
        self.gd = gd
        self.regression = regression
        self.stochastic = stochastic
        self.batch_size = batch_size
        self.regularize = regularize
        self.regLambda = regLambda
    
    # divide data into training and testing samples
    def trainTestSplit(self, X, y):
        """
        @X: numpy matrix, dataset
        @y: numpy array, target value
        @does: splitting dataset to train and test 
        @return: numpy matrix, matrix, array, array
        """
        X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
        return X_train, X_test, Y_train, Y_test
    
    def addX0(self, X):
        """
        @X: numpy matrix, dataset
        @does: add a bias term to the data
        @return: numpy matrix
        """
        return np.column_stack([np.ones([X.shape[0], 1]), X])
    
    def normalize(self, X):
        """
        @X: numpy matrix, dataset
        @does: normalize the dataset
        @return: numpy matrix, array, array
        """
        
        mean = np.mean(X, 0) # mean of each column defined by the 0
        std = np.std(X,0)
        X_norm = (X - mean)/std
        X_norm = self.addX0(X_norm)
        
        return X_norm, mean, std # we will be normalizing the test data based on the mean and std of training data
    
    def normalizeTestData(self, X, trainMean, trainStd):
        """
        @X: numpy matrix, dataset
        @does: normalize the test dataset
        @return: numpy matrix
        """
        
        X_norm = (X - trainMean)/trainStd
        X_norm = self.addX0(X_norm)
        
        return X_norm
    
    def rank(self, X, eps = 1e-12):
        """
        @X: numpy matrix
        @eps: float
        @does: return rank of the matrix
        @return: numpy matrix
        """
        u, S, vh = np.linalg.svd(X)
        return len([x for x in S if abs(x) > eps])
    
    def checkMatrix(self, X):
        """
        @X: numpy matrix
        @does: check if the matrix is full rank
        @return
        """
        x_rank = self.rank(X)
        if x_rank == min(X.shape[0], X.shape[1]):
            self.fullRank = True
            print("Matrix is full rank")
        else:
            self.fullRank = False
            print("Matrix is not full rank")
            
    def checkInvertibility(self, X):
        """
        @X: numpy matrix
        @does: check if the matrix is low rank
        @return
        """
        if X.shape[0] < X.shape[1]:
            self.lowRank = True
            print('The matrix is low rank')
        else:
            self.lowRank = False
            
    def closedFormSolution(self, X, y):
        """
        @X: numpy matrix, dataset
        @y: numpy array, target value
        @does: solve regression using closed form solution 
        @return: numpy array - parameters theta
        """
        return np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
    
    def predict(self, X):
        """
        @X: numpy matrix, dataset
        @does: compute prediction 
        @return: numpy array - prediction array
        """
        return X.dot(self.w)
    
    def sse(self, X, y): #standard square error
        """
        @X: numpy matrix, dataset
        @y: numpy array, target value
        @does: compute sum of squared error 
        @return: integer - overall SSE
        """
        y_hat = self.predict(X)
        return ((y_hat - y)**2).sum()
    
    def rmse(self, X, y):
        """
        @X: numpy matrix, dataset
        @y: numpy array, target value
        @does: compute root mean square error 
        @return: integer - overall RMSE
        """
        return math.sqrt(self.sse(X,y) / y.size)
    
    def costFunction(self, X, y):
        """
        @X: numpy matrix, dataset
        @y: numpy array, target value
        @does: compute root mean square error 
        @return: integer - overall cost
        """
        return self.sse(X,y)/2
    
    def costDerivative(self, X, y):
        """
        @X: numpy matrix, dataset
        @y: numpy array, target value
        @does: computes derivative of the cost function 
        @return: integer - derivative value
        """
        y_hat = self.predict(X)
        return (y_hat - y).dot(X) #2X.T*Theta*X - 2X.T*y

    def gradientDescent(self, X, y):
        """
        @X: numpy matrix, dataset
        @y: numpy array, target value
        @does: normal gradient descent and stochastic gradient descent
        @return
        """
        error_sequences = []
        previous_error = float('inf')
        
        # Stochastic Gradient Descent
        
        if self.stochastic:
            print("Stochastic Gradient Descent")
            for i in tqdm(range(self.maxIteration)):
                idx = np.random.randint(0, X.shape[0], self.batch_size) #randomly generate indexes
                X_batch = X[idx, :] #randomly sample from X
                y_batch = y[idx] #randomly sample from y
                self.w = self.w - self.learningrate * self.costDerivative(X_batch, y_batch)
                if self.error == 'rmse':
                    current_error = self.rmse(X, y)
                else:
                    current_error = self.sse(X, y)
                diff = previous_error - current_error
                previous_error = current_error
                if diff < self.tolerance:
                    print("No further imporvements")
                    break
                    
        else:
            print("Normal Gradient Descent")
            for i in tqdm(range(self.maxIteration)):
                self.w = self.w - self.learningrate * self.costDerivative(X, y)
                if self.error == 'rmse':
                    current_error = self.rmse(X, y)
                else:
                    current_error = self.sse(X, y)
                    
                diff = previous_error - current_error
                previous_error = current_error
                if diff < self.tolerance:
                    print("No further imporvement")
                    break
        return
    
    
    # L2 Regularization 
    
    def regCostDerivative(self, X, y):
        """
        @X: numpy matrix, dataset
        @y: numpy array, target value
        @does: computes cost derivative for L2 regularization 
        @return: integer - overall cost
        """
        return self.costDerivative(X, y) + self.regLambda * self.w
     
    def RidgeRegularization(self, X, y):
        """
        @X: numpy matrix, dataset
        @y: numpy array, target value
        @does: computes w for Ridge Regularization using both closed form and grdient descent
        @return: integer - overall cost
        """
        if self.gd:
            print("Regularization - Gradient Descent")
            previous_error = float('inf')
            for i in tqdm(range(self.maxIteration)):
                self.w = self.w - self.learningrate * self.regCostDerivative(X, y)
                if self.error == 'rmse':
                    current_error = self.rmse(X, y)
                else:
                    current_error = self.sse(X, y)
                    
                diff = previous_error - current_error
                previous_error = current_error
                if diff < self.tolerance:
                    print("No further imporvement")
                    break
        else:
            print("Regularization - Closed Form Solution")
            I = np.identity(X.shape[1])
            self.w = np.linalg.inv(X.T.dot(X) + self.regLambda*I).dot(X.T).dot(y)
        
    
    def trainModel(self):
        """
        @does: combines above methods to train model
        """
        # Split dataset to train and test
        self.X_train, self.X_test, self.y_train, self.y_test = self.trainTestSplit(self.X, self.y)
        
        # Normalize Data
        self.X_train, self.mean, self.std = self.normalize(self.X_train)
        self.X_test = self.normalizeTestData(self.X_test, self.mean, self.std)
        
        # Check matrix rank
        self.checkMatrix(self.X_train)
        self.checkInvertibility(self.X_train)
        
        # No Regularization
        if not self.regularize:
            # Closed Form Solution
            if self.fullRank and not self.lowRank and self.X_train.shape[0] < 10000 and not self.gd:
                print("Normal Closed Form Solution")
                self.w = self.closedFormSolution(self.X_train, self.y_train)
            
            # Gradient Descent
            else:
                self.w = np.ones(self.X_train.shape[1], dtype = np.float64) * 0
                self.gradientDescent(self.X_train, self.y_train)
        
        # With Regularization
        else:
            self.w = np.ones(self.X_train.shape[1], dtype = np.float64) * 0
            self.RidgeRegularization(self.X_train, self.y_train)
            
            
        print(self.w)
        
        if self.error == 'rmse':
            print(self.rmse(self.X_test, self.y_test))
        else:
            print(self.sse(self.X_test, self.y_test))

In [285]:
regression  = LinearRegression(housing.values[:, 0:-1], housing.values[:,-1], 
                              learningrate = 0.000001,
                              tolerance = 0.0000001,
                              gd = True,
                              error = 'rmse',
                              stochastic = False,
                              batch_size = 150,
                              regularize = True,
                              regLambda = 1)

In [286]:
regression.trainModel()

  3%|██▍                                                                       | 1627/50000 [00:00<00:02, 16170.43it/s]

Matrix is full rank
Regularization - Gradient Descent


100%|█████████████████████████████████████████████████████████████████████████| 50000/50000 [00:02<00:00, 16708.72it/s]

[22.86920857 -0.85100296  0.93948444  0.14432545  0.69565732 -1.96577407
  2.67322031  0.3778522  -2.53847105  1.65766706 -1.01910805 -2.27888403
  0.65303712 -3.92456451]
4.638097616704881


In [282]:
concrete = pd.read_csv("concreteData.csv")
concrete.head()

,540,0,0.1,162,2.5,1040,676,28,79.99
0,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
1,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
3,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
4,266.0,114.0,0.0,228.0,0.0,932.0,670.0,90,47.03


In [287]:
regression_concrete  = LinearRegression(concrete.values[:, 0:-1], concrete.values[:,-1], 
                              learningrate = 0.000001,
                              tolerance = 0.0000001,
                              gd = True,
                              error = 'rmse',
                              stochastic = False,
                              batch_size = 150,
                              regularize = True,
                              regLambda = 1)

In [289]:
regression_concrete.trainModel()

  2%|█▏                                                                          | 810/50000 [00:00<00:06, 7952.89it/s]

Matrix is full rank
Regularization - Gradient Descent


100%|█████████████████████████████████████████████████████████████████████████| 50000/50000 [00:03<00:00, 13446.48it/s]

[35.50937587 11.08940232  7.29592748  4.34045552 -4.01296937  1.95870087
  0.22722463 -0.14746242  7.29396228]
11.104586195591038


In [291]:
yacht = pd.read_csv("yachtData.csv")
yacht.head()

,-2.3,0.568,4.78,3.99,3.17,0.125,0.11
0,-2.3,0.568,4.78,3.99,3.17,0.150,0.27
1,-2.3,0.568,4.78,3.99,3.17,0.175,0.47
2,-2.3,0.568,4.78,3.99,3.17,0.200,0.78
3,-2.3,0.568,4.78,3.99,3.17,0.225,1.18
4,-2.3,0.568,4.78,3.99,3.17,0.250,1.82


In [293]:
regression_yacht  = LinearRegression(yacht.values[:, 0:-1], yacht.values[:,-1], 
                              learningrate = 0.000001,
                              tolerance = 0.0000001,
                              gd = True,
                              error = 'rmse',
                              stochastic = False,
                              batch_size = 150,
                              regularize = True,
                              regLambda = 1)

In [294]:
regression_yacht.trainModel()

  4%|██▋                                                                       | 1822/50000 [00:00<00:02, 17562.58it/s]

Matrix is full rank
Regularization - Gradient Descent


 57%|█████████████████████████████████████████▊                               | 28616/50000 [00:01<00:00, 24865.72it/s]

No further imporvement
[10.67000694  0.53987566 -0.64163478 -0.11601129 -0.06174425 -0.22182343
 12.8031058 ]
9.025975982731234
